In [ ]:
! pip install sentencepiece

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00


In [ ]:
! pip install transformers
! pip install datasets
! pip install rouge_score
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5d54e2b0ea49ed2e3dae4535a7986d8a345df6d8f2632e5329e931bcdf157de0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7

In [ ]:
from datasets import Dataset

In [ ]:
model_name="abdalrahmanshahrour/arabartsummarization"

In [ ]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

wandb: Currently logged in as: fady-maher2024 (tech-pharaohs). Use `wandb login --relogin` to force relogin


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set model parameters or use the default
# print(model.config)

# tokenization
encoder_max_length = 256  # demo
decoder_max_length = 64

In [ ]:
print(model.config)

MBartConfig {
  "_name_or_path": "abdalrahmanshahrour/arabartsummarization",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": 

In [ ]:
dataset = datasets.load_dataset("csebuetnlp/xlsum", name="arabic", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/arabic/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 37519
})

In [ ]:
dataset = dataset.remove_columns(["id","url","title"])

In [ ]:
train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

In [ ]:
dataset

Dataset({
    features: ['summary', 'text'],
    num_rows: 37519
})

In [ ]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    #source, target = batch["document"], batch["summary"]
    source, target = batch["text"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)


#validation_data = dataset.map(
 #   lambda batch: batch_tokenize_preprocess(
  #      batch, tokenizer, encoder_max_length, decoder_max_length
   # ),
    #batched=True,
    #remove_columns=dataset.column_names,
#)

Map:   0%|          | 0/33767 [00:00<?, ? examples/s]

Map:   0%|          | 0/3752 [00:00<?, ? examples/s]

In [ ]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-22-16757958f45a>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    seed=42,
    learning_rate=5e-05,
    warmup_steps=250,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
    lr_scheduler_type="linear",
    optim="adamw_hf",
    adam_beta1 = 0.9,
    adam_beta2= 0.999,
    adam_epsilon= 1e-08,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

In [ ]:
language = "arabic"

In [ ]:
#final
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


You're using a BarthezTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,4.114000
100,3.893900
150,3.911500
200,3.829500
250,3.893200
300,3.907800
350,3.886900
400,3.930800
450,3.954800
500,3.859900


TrainOutput(global_step=42210, training_loss=3.4021675071454225, metrics={'train_runtime': 9422.5447, 'train_samples_per_second': 17.918, 'train_steps_per_second': 4.48, 'total_flos': 2.573701039521792e+16, 'train_loss': 3.4021675071454225, 'epoch': 5.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_dir = '/content/drive/MyDrive/arabartsummarization/'
trainer.save_model(model_dir + 'model2')

In [ ]:
# Save the fine-tuned model to Google Drive
model.save_pretrained("/content/drive/MyDrive/arabartsummarization/")

In [ ]:
trainer.evaluate()

{'eval_loss': 3.5774919986724854,
 'eval_rouge1': 1.9287,
 'eval_rouge2': 0.1333,
 'eval_rougeL': 1.9176,
 'eval_rougeLsum': 1.9092,
 'eval_gen_len': 19.681,
 'eval_runtime': 356.6942,
 'eval_samples_per_second': 10.519,
 'eval_steps_per_second': 2.63,
 'epoch': 5.0}

Validation

In [ ]:
summaries_before_tuning

['قتل أربعة أشخاص، على الأقل، وجرح ثمانية آخرون، في انفجار عبوة ناسفة، استهدفت',
 'اعتذرت شركة "بجي" المصرية لألعاب الفيديو عن خاصية جديدة في لعبة "ب',
 'أثارت حادثة أضرم طالب أردني النار في نفسه غضبا واسعا عبر مواقع التواصل الاجتماعي في الأردن',
 'بدأ علماء ليبيريون تجربة لقاح ضد فيروس إيبولا على متطوعين في دولة ليبيريا',
 'كشفت شركة "تشارج" البريطانية لصناعة السيارات عن شاحنة كهربائية تعمل بالطاقة الذاتية القيادة،',
 'تفيد الأنباء الواردة من سوريا بوقوع اشتباكات عنيفة بين قوات كردية ومسلحي تنظيم الدولة الإسلامية',
 'قال رئيس الائتلاف الوطني السوري المعارض، أحمد معاذ الخطيب، إنه لا يزال متمسكا بالحوار',
 'حذر فريق بحثي من أن تناول كميات كبيرة من الطعام قد يزيد من خطر الإصابة بأمراض',
 'وافق المجلس الأعلى للقضاء في مصر على ندب القضاة وأعضاء النيابة العامة للإشراف على الاستفتاء على مشروع',
 'قال رئيس المؤسسة الوطنية للنفط في ليبيا، مصطفى صنع الله، إن ميناء راس لانوف',
 'أسفرت قرعة دور الستة عشر لدوري أبطال أوروبا لكرة القدم عن وقوع نادي ريال مدريد الإسباني في',
 'أعرب البابا فرنسيس عن س

In [ ]:
summaries_after_tuning

['قتل أربعة أشخاص، على الأقل، وجرح ثمانية آخرون، في انفجار عبوة ناسفة، زرعت',
 'اعتذرت شركة ببجي للهواتف الذكية عن إضافة خاصية الأصنام في لعبة "ببجي',
 'أثارت حادثة أضرم طالب أردني النار في نفسه داخل جامعة الملك عبد الله الثاني في العاصمة',
 'بدأ علماء في ليبيريا تجربة للقاح ضد فيروس إيبولا في محاولة للسيطرة على المرض.',
 'كشفت شركة "تشارج" المصنعة لشاحنات التوصيل عن أول شاحنة كهربائية ذاتية القيادة في',
 'تفيد الأنباء الواردة من سوريا بوقوع اشتباكات عنيفة بين مسلحي تنظيم الدولة الإسلامية ومسلحي الميليشيات',
 'قال رئيس الائتلاف الوطني السوري المعارض، أحمد معاذ الخطيب، إنه لا يزال يرغب في الحوار',
 'حذر فريق بحثي من أن تناول كميات كبيرة من الطعام قد يزيد من خطر الإصابة بأمراض',
 'وافق المجلس الأعلى للقضاء في مصر على انتداب القضاة وأعضاء النيابة العامة للإشراف على الاستفتاء على مشروع',
 'قال رئيس المؤسسة الوطنية للنفط في ليبيا، مصطفى صنع الله، إن ميناء راس لانوف',
 'أسفرت قرعة دور الستة عشر في دوري أبطال أوروبا عن وقوع ناديي ريال مدريد الإسباني و',
 'أعرب البابا فرنسيس عن سعادته بفوز ناد

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

In [ ]:
results = rouge.get_scores(summaries_after_tuning, test_samples['summary'])

In [ ]:
results

In [ ]:
import pandas as pd
from zipfile import ZipFile
from typing import Dict, List
import random
import os
from tempfile import TemporaryDirectory

In [ ]:
val_data = pd.read_json("/content/drive/MyDrive/AI_competition/validation_data.jsonl", lines=True)

In [ ]:
val_data

,example_id,paragraph
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...
...,...,...
267,267,وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...
268,268,كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...
269,269,حشدت فرنسا قواتها وأعلنت الحرب في 19 يوليو أي ...
270,270,تعتبر قناة السويس هي من أهم الطرق الملاحية الت...


In [ ]:
dataset = Dataset.from_pandas(val_data)

In [ ]:
dataset

Dataset({
    features: ['example_id', 'paragraph'],
    num_rows: 272
})

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        #test_samples["document"],
        #test_samples["text"],
        test_samples,
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = dataset["paragraph"][250:272]

#summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

In [ ]:
summaries_after_tuning

In [ ]:
submission = pd.DataFrame()

In [ ]:
submission = pd.DataFrame(val_data['example_id'], columns=['example_id'])

In [ ]:
submission

,example_id
0,0
1,1
2,2
3,3
4,4
...,...
267,267
268,268
269,269
270,270


In [ ]:
summary = []

In [ ]:
for i in range(len(summaries_after_tuning)):
  summary.append(summaries_after_tuning[i])

In [ ]:
len(summary)

272

In [ ]:
submission['summary'] = summary

In [ ]:
submission

,example_id,summary
0,0,فتح بيت المقدس، ثاني أكبر المدن الإسلامية في ا...
1,1,في مثل هذا الوقت من عام 662، دعا عمر بن الخطاب...
2,2,"في مثل هذا الوقت من كل عام يحتفل المسلمون ب "" ..."
3,3,شهدت منطقة شمال إفريقيا في العصر الأموي عددا م...
4,4,في مثل هذا الوقت من عام 1453 ه، قامت دولة الخل...
...,...,...
267,267,في العصر العباسي الأول ، كان الاهتمام منصبا عل...
268,268,في الثلاثينيات من القرن الماضي ، كانت صناعة ال...
269,269,في مثل هذا الشهر من عام 1870 اندلعت الحرب الفر...
270,270,تعتبر قناة السويس من أهم الطرق الملاحية التي ي...


In [ ]:
submission_dictionary = { example_id : summary for _, example_id, summary in submission[['example_id', 'summary']].itertuples() }
submission_dictionary

{0: 'فتح بيت المقدس، ثاني أكبر المدن الإسلامية في العالم الإسلامي، بعد مكة والمدينة المنورة،',
 1: 'في مثل هذا الوقت من عام 662، دعا عمر بن الخطابة بطريرك بيت المقدس',
 2: 'في مثل هذا الوقت من كل عام يحتفل المسلمون ب " الفتح العظيم" لمنطقة النوبة',
 3: 'شهدت منطقة شمال إفريقيا في العصر الأموي عددا من الغزوات الإسلامية، وكان من أبرزها فتح',
 4: 'في مثل هذا الوقت من عام 1453 ه، قامت دولة الخلافة الأموية بقيادة الأموي',
 5: 'شهد العصر الأموي عددا من الغزوات والهجمات التي قام بها الراشدون في',
 6: 'هل تعرف أسباب فشل المسلمين في حصار مدينة القسطنطينية؟',
 7: 'بعد أن فتح "معاوية بن أبي سفيان" إفريقية في عهد "الوليد بن',
 8: 'بعد أن استكمل المسلمون فتح شمال إفريقيا واستقرار أمور الحكم بها، تطلعت القبائل العربية بجوار',
 9: 'في مثل هذا الوقت من عام 1492، توجه القائد الإسلامي موسى بن نصير إلى الأندلس',
 10: 'في مثل هذا الوقت من عام ١١٤ه  ٧٣٢م،',
 11: 'في مثل هذا الوقت من عام (١٣٢ ه  ٧٥٠',
 12: 'هل تعرف كيف تحقق الفتح الإسلامي في بلاد الهند والسند؟',
 13: 'استولى العباسيون على أجزاء من البحر الم

In [ ]:
# import pandas as pd

# submission = pd.DataFrame()
# submission['example_id'] = val_data['example_id']  # Set the 'example_id' column in the DataFrame

# summary = []
# for i in range(len(summaries_after_tuning)):
#     summary.append(summaries_after_tuning[i])

# submission['summary'] = summary  # Set the 'summary' column in the DataFrame

# submission_dictionary = submission.set_index('example_id').to_dict()['summary']


In [ ]:
import pandas as pd
from zipfile import ZipFile
from typing import Dict, List
import random
import os
from tempfile import TemporaryDirectory

def create_submission(output_file_path : str, submission_dictionary : Dict[int, str], base_keys : List[int]) -> None:
    """Function that validates the submission data types and schema and zip it to be ready from submission

    Parameters
    ----------
    output_file_path : str
        The locaiton and file name you want to save the zip file at, ex : "/home/user/submission_123.zip"
    submission_dictionary : dict[int, str]
        dictionary of int keys (example_id) and string values (summary)
    base_keys: list[int]
        list of keys of the original unlabeled validation set


    Returns
    -------
    None
    """
    #assertions
    assert all(isinstance(i, int) for i in submission_dictionary.keys()), "Make sure example_ids elements (key of submission_dictionary) are of type int"
    assert all(isinstance(i, str) for i in submission_dictionary.values()), "Make sure summary elements (value of submission_dictionary) are of type str"
    assert all(isinstance(i, int) for i in base_keys), "Make sure base_keys elements is of type int"

    diff_sub = set(submission_dictionary.keys()) - set(base_keys)
    diff_base = set(base_keys) - set(submission_dictionary.keys())

    assert len(diff_sub) == 0, f"Keys {diff_sub} is in submission but not in base_keys"
    assert len(diff_base) == 0, f"Keys {diff_base} is in base_keys but not in submission"

    #saving
    final_submission = pd.DataFrame(submission_dictionary.items(), columns=['example_id', 'summary'])

    if final_submission.example_id.dtype != 'int64' :
        final_submission.example_id = final_submission.example_id.astype(int)

    assert len(final_submission[final_submission.summary.isna()]) == 0, f"summaries with the example_id = {final_submission[final_submission.summary.isna()].example_id.values.tolist()} is NaN"
    assert len(final_submission[final_submission.example_id.isna()]) == 0, f"example_ids with the following index = {final_submission[final_submission.example_id.isna()].index.tolist()} is NaN"

    with TemporaryDirectory(dir=".") as tmpdirname:
        os.chdir(tmpdirname)
        jsonl_name = "predictions.jsonl"
        final_submission.to_json(jsonl_name, lines=True, orient='records', force_ascii=False)
        with ZipFile(output_file_path, "w") as zip_file:
            zip_file.write(filename = jsonl_name)
            print(f"Submission of {jsonl_name} as .zip saved at {output_file_path}")
        os.chdir("..")

In [ ]:
base_keys = val_data.example_id.values.tolist()
len(base_keys)

272

In [ ]:
create_submission(output_file_path = "/content/drive/MyDrive/AI_competition/first_submission.zip", submission_dictionary = submission_dictionary, base_keys = base_keys)

Submission of predictions.jsonl as .zip saved at /content/drive/MyDrive/AI_competition/first_submission.zip
